In [40]:
### Packages
! pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [41]:
### Print functions
def solve_print():
    m.reset()
    m.optimize()
    if m.Status == GRB.OPTIMAL:
        print("Total Profit = {0:.2f} ".format(m.ObjVal))
        f.write("Total Profit, {}".format(m.ObjVal))
        csv_out = np.zeros([m.numvars,5])
        row_label = []
        for var in m.getVars(): row_label.append(var.varname)
        column_label = ['X', 'Obj', 'rc', 'Objlow','ObjUp']

        tp=0
        for var in m.getVars():
            csv_out[tp,0] = var.x
            csv_out[tp,1] = var.obj
            csv_out[tp,2] = var.rc
            csv_out[tp,3] = var.saobjlow
            csv_out[tp,4] = var.saobjup
            tp += 1
        df = pd.DataFrame(csv_out, columns=column_label, index=row_label)
        f.write('\n')
        df.to_csv(f,mode='a')
        m.printAttr(['X', 'Obj', 'rc', 'saobjlow', 'saobjup'])
        
        csv_out = np.zeros([m.numconstrs,5])
        row_label = []
        for constr in m.getConstrs(): row_label.append(constr.constrname)
        column_label = ['RHS', 'Slack','pi', 'RHSlow','RHSup']

        tp = 0
        for const in m.getConstrs():
            csv_out[tp,0] = const.RHS
            csv_out[tp,1] = const.slack
            csv_out[tp,2] = const.pi
            csv_out[tp,3] = const.sarhslow
            csv_out[tp,4] = const.sarhsup
            tp += 1   
        df = pd.DataFrame(csv_out, columns=column_label, index=row_label)
        f.write("\n")
        df.to_csv(f,mode='a')
        f.write("\n")
        m.printAttr(['RHS', 'Sense', 'Slack', 'pi', 'sarhslow', 'sarhsup' ])

    else: print('Could not solve model')

def solve_print_console():
    m.reset()
    m.optimize()
    if m.Status == GRB.OPTIMAL:
        print("Total Cost = {0:.4f} ".format(m.ObjVal))
        m.printAttr(['X', 'Obj', 'rc', 'saobjlow', 'saobjup'])
        m.printAttr(['RHS', 'Sense', 'Slack', 'pi', 'sarhslow', 'sarhsup' ])
    else: print("Could not solve model")
    
    
### Sensitivity functions    
def set_obj (i, j):
    m.getVars()[i].obj = j

    
def set_rhs(i,j):
    m.getConstrs()[i].rhs=j    
    
def new_obj_calc(i,j):
    save = m.getVars()[i].obj
    if j>0: new = objup[i] + epsilon
    else: new = objlow[i] - epsilon
    print("\n\n New Obj ",m.getVars()[i].VarName," = ",new)
    set_obj(i,new)
    solve_print()
    set_obj(i,save)
    m.update()

    
def new_rhs_calc(i,j):
    save = m.getConstrs()[i].rhs
    if j>0: new = rhsup[i] + epsilon
    else: new = rhslow[i] - epsilon 
    print("\n\n New RHS ",m.getConstrs()[i].ConstrName," = ",new)
    set_rhs(i,new)
    solve_print()
    set_rhs(i,save)
    m.update()

    
def solve_print_console():
    m.reset()
    m.optimize()
    if m.Status == GRB.OPTIMAL:
        print("Total Profit = {0:.2f} ".format(m.ObjVal))
        m.printAttr(['X', 'Obj', 'rc', 'saobjlow', 'saobjup'])
        m.printAttr(['RHS', 'Sense', 'Slack', 'pi', 'sarhslow', 'sarhsup' ])
    else: print('Could not solve model')
    
    
### Sensitivity functions
def set_obj (i, j):
    m.getVars()[i].obj = j

    
def set_rhs(i,j):
    m.getConstrs()[i].rhs=j    
    
def new_obj_calc(i,j):
    save = m.getVars()[i].obj
    if j>0: new = objup[i] + epsilon
    else: new = objlow[i] - epsilon
    print("\n\n New Obj ",m.getVars()[i].VarName," = ",new)
    set_obj(i,new)
    solve_print()
    set_obj(i,save)
    m.update()

    
def new_rhs_calc(i,j):
    save = m.getConstrs()[i].rhs
    if j>0: new = rhsup[i] + epsilon
    else: new = rhslow[i] - epsilon 
    print("\n\n New RHS ",m.getConstrs()[i].ConstrName," = ",new)
    set_rhs(i,new)
    solve_print()
    set_rhs(i,save)
    m.update()

In [63]:
########################################
### Main logic
########################################
epsilon = 0.0001
inf = 1e30
f = open("temp.csv","w", newline='')

m = gp.Model("Critters Product")
m.ModelSense = GRB.MAXIMIZE

In [64]:
###
### Variables
###

#ad stands for additional 
#T stands for treatment
#1 and 2 stands for Agent 1 and 2
#A and B stands for envoronment A and B

num_vars = 12

Return = np.array([-11, -17.5, -15, -21.5, 82, 87.5, 82, 87.5, 75.5, 81, 75.5, 81])
var_names = (['A','B','A_ad','B_ad','T_A1','T_A2','T_B1','T_B2','T_A1_ad','T_A2_ad','T_B1_ad','T_B2_ad'])

x = m.addMVar(num_vars, obj=Return, name=var_names)
m.update()

In [53]:
#########constraints

#environment constraint
A = np.zeros([1,num_vars])
A[0,0]=1
A[0,1]=1
rhs = [1000]
m.addMConstr(A , x, '<', rhs, name="Environment")
m.update()

#aditional environment constraint
A = np.zeros([1,num_vars])
A[0,2]=1
A[0,3]=1
rhs = [400]
m.addMConstr(A , x, '<', rhs, name="Additional Environment")
m.update()

#Bake treatment
A = np.zeros([1,num_vars])
for i in range(4,8):
    A[0,i]=1
rhs = [800]
m.addMConstr(A , x, '<', rhs, name="Bake Treatment")
m.update()

#Additional bake treatment
A = np.zeros([1,num_vars])
for i in range(8,12):
    A[0,i]=1
rhs = [300]
m.addMConstr(A , x, '<', rhs, name="Additional Bake Treatment")
m.update()

#Percent good critter
A = np.zeros([1,num_vars])
A[0,4]=0.1
A[0,8]=0.1
A[0,6]=0.2
A[0,10]=0.2
A[0,7]=-0.1
A[0,11]=-0.1
rhs = [0]
m.addMConstr(A , x, '>', rhs, name="Good Critter Percentage")
m.update()


#Percent bad critter
A = np.zeros([1,num_vars])
A[0,4]=-0.02
A[0,8]=-0.02
A[0,5]=-0.01
A[0,9]=-0.01
A[0,6]=0.03
A[0,10]=0.03
A[0,7]=-0.01
A[0,11]=-0.01
rhs = [0]
m.addMConstr(A , x, '<', rhs, name="Bad Critter Percentage")
m.update()

#environmnet A harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,0]=-0.4
A[0,2]=-0.4
A[0,4]=1
A[0,5]=1
A[0,8]=1
A[0,9]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment A relationship with mix treated")
m.update()

#environmnet B harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,1]=-0.7
A[0,3]=-0.7
A[0,6]=1
A[0,7]=1
A[0,10]=1
A[0,11]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment B relationship with mix treated")
m.update()

In [54]:
solve_print()

Discarded solution information
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 38 nonzeros
Model fingerprint: 0xf3cf6b35
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve time: 0.02s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5200000e+32   1.600000e+31   6.520000e+02      0s
       9    5.2933617e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.03 seconds (0.00 work units)
Optimal objective  5.293361702e+04
Total Profit = 52933.62 

    Variable            X          Obj           rc     saobjlow      saobjup 
-----------------------------------------------------------------------------
           A      208.511          -11            0      

In [55]:
# save the original problem details before sensitivity
objup = m.getAttr('SaObjUp')
objlow = m.getAttr('SaObjlow')
rhsup = m.getAttr('SaRhsUp')
rhslow = m.getAttr('SaRhsLow')

In [60]:
#change A harvest yeild by 1 percent (tried up to 50%)
#########constraints

#environment constraint
A = np.zeros([1,num_vars])
A[0,0]=1
A[0,1]=1
rhs = [1000]
m.addMConstr(A , x, '<', rhs, name="Environment")
m.update()

#aditional environment constraint
A = np.zeros([1,num_vars])
A[0,2]=1
A[0,3]=1
rhs = [400]
m.addMConstr(A , x, '<', rhs, name="Additional Environment")
m.update()

#Bake treatment
A = np.zeros([1,num_vars])
for i in range(4,8):
    A[0,i]=1
rhs = [800]
m.addMConstr(A , x, '<', rhs, name="Bake Treatment")
m.update()

#Additional bake treatment
A = np.zeros([1,num_vars])
for i in range(8,12):
    A[0,i]=1
rhs = [300]
m.addMConstr(A , x, '<', rhs, name="Additional Bake Treatment")
m.update()

#Percent good critter
A = np.zeros([1,num_vars])
A[0,4]=0.1
A[0,8]=0.1
A[0,6]=0.2
A[0,10]=0.2
A[0,7]=-0.1
A[0,11]=-0.1
rhs = [0]
m.addMConstr(A , x, '>', rhs, name="Good Critter Percentage")
m.update()


#Percent bad critter
A = np.zeros([1,num_vars])
A[0,4]=-0.02
A[0,8]=-0.02
A[0,5]=-0.01
A[0,9]=-0.01
A[0,6]=0.03
A[0,10]=0.03
A[0,7]=-0.01
A[0,11]=-0.01
rhs = [0]
m.addMConstr(A , x, '<', rhs, name="Bad Critter Percentage")
m.update()

#environmnet A harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,0]=-0.5
A[0,2]=-0.5
A[0,4]=1
A[0,5]=1
A[0,8]=1
A[0,9]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment A relationship with mix treated")
m.update()

#environmnet B harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,1]=-0.7
A[0,3]=-0.7
A[0,6]=1
A[0,7]=1
A[0,10]=1
A[0,11]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment B relationship with mix treated")
m.update()

In [61]:
solve_print()

Discarded solution information
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 38 nonzeros
Model fingerprint: 0x3bcd2297
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve time: 0.01s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5200000e+32   1.600000e+31   6.520000e+02      0s
      12    5.4921053e+04   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.03 seconds (0.00 work units)
Optimal objective  5.492105263e+04
Total Profit = 54921.05 

    Variable            X          Obj           rc     saobjlow      saobjup 
-----------------------------------------------------------------------------
           A            0          -11           -0     

In [65]:
#change B harvest yeild by 1 percent (tried up to 75%)
#########constraints

#environment constraint
A = np.zeros([1,num_vars])
A[0,0]=1
A[0,1]=1
rhs = [1000]
m.addMConstr(A , x, '<', rhs, name="Environment")
m.update()

#aditional environment constraint
A = np.zeros([1,num_vars])
A[0,2]=1
A[0,3]=1
rhs = [400]
m.addMConstr(A , x, '<', rhs, name="Additional Environment")
m.update()

#Bake treatment
A = np.zeros([1,num_vars])
for i in range(4,8):
    A[0,i]=1
rhs = [800]
m.addMConstr(A , x, '<', rhs, name="Bake Treatment")
m.update()

#Additional bake treatment
A = np.zeros([1,num_vars])
for i in range(8,12):
    A[0,i]=1
rhs = [300]
m.addMConstr(A , x, '<', rhs, name="Additional Bake Treatment")
m.update()

#Percent good critter
A = np.zeros([1,num_vars])
A[0,4]=0.1
A[0,8]=0.1
A[0,6]=0.2
A[0,10]=0.2
A[0,7]=-0.1
A[0,11]=-0.1
rhs = [0]
m.addMConstr(A , x, '>', rhs, name="Good Critter Percentage")
m.update()


#Percent bad critter
A = np.zeros([1,num_vars])
A[0,4]=-0.02
A[0,8]=-0.02
A[0,5]=-0.01
A[0,9]=-0.01
A[0,6]=0.03
A[0,10]=0.03
A[0,7]=-0.01
A[0,11]=-0.01
rhs = [0]
m.addMConstr(A , x, '<', rhs, name="Bad Critter Percentage")
m.update()

#environmnet A harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,0]=-0.4
A[0,2]=-0.4
A[0,4]=1
A[0,5]=1
A[0,8]=1
A[0,9]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment A relationship with mix treated")
m.update()

#environmnet B harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,1]=-0.75
A[0,3]=-0.75
A[0,6]=1
A[0,7]=1
A[0,10]=1
A[0,11]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment B relationship with mix treated")
m.update()

In [67]:
solve_print()

Discarded solution information
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 38 nonzeros
Model fingerprint: 0xd81b1662
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve time: 0.02s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5200000e+32   1.600000e+31   6.520000e+02      0s
       9    5.7374737e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.03 seconds (0.00 work units)
Optimal objective  5.737473684e+04
Total Profit = 57374.74 

    Variable            X          Obj           rc     saobjlow      saobjup 
-----------------------------------------------------------------------------
           A      221.053          -11            0      

In [72]:
########################################full cost staff 
epsilon = 0.0001
inf = 1e30
f = open("temp.csv","w", newline='')

m = gp.Model("Critters Product")
m.ModelSense = GRB.MAXIMIZE

In [73]:
###
### Variables
###

#ad stands for additional 
#T stands for treatment
#1 and 2 stands for Agent 1 and 2
#A and B stands for envoronment A and B

num_vars = 12

Return = np.array([-59, -52.5, -25, -18.5, 62, 67.5, 62, 67.5, 35.5, 41, 35.5, 41])
var_names = (['A','B','A_ad','B_ad','T_A1','T_A2','T_B1','T_B2','T_A1_ad','T_A2_ad','T_B1_ad','T_B2_ad'])

x = m.addMVar(num_vars, obj=Return, name=var_names)
m.update()

In [74]:
#########constraints

#environment constraint
A = np.zeros([1,num_vars])
A[0,0]=1
A[0,1]=1
rhs = [1000]
m.addMConstr(A , x, '<', rhs, name="Environment")
m.update()

#aditional environment constraint
A = np.zeros([1,num_vars])
A[0,2]=1
A[0,3]=1
rhs = [400]
m.addMConstr(A , x, '<', rhs, name="Additional Environment")
m.update()

#Bake treatment
A = np.zeros([1,num_vars])
for i in range(4,8):
    A[0,i]=1
rhs = [800]
m.addMConstr(A , x, '<', rhs, name="Bake Treatment")
m.update()

#Additional bake treatment
A = np.zeros([1,num_vars])
for i in range(8,12):
    A[0,i]=1
rhs = [300]
m.addMConstr(A , x, '<', rhs, name="Additional Bake Treatment")
m.update()

#Percent good critter
A = np.zeros([1,num_vars])
A[0,4]=0.1
A[0,8]=0.1
A[0,6]=0.2
A[0,10]=0.2
A[0,7]=-0.1
A[0,11]=-0.1
rhs = [0]
m.addMConstr(A , x, '>', rhs, name="Good Critter Percentage")
m.update()


#Percent bad critter
A = np.zeros([1,num_vars])
A[0,4]=-0.02
A[0,8]=-0.02
A[0,5]=-0.01
A[0,9]=-0.01
A[0,6]=0.03
A[0,10]=0.03
A[0,7]=-0.01
A[0,11]=-0.01
rhs = [0]
m.addMConstr(A , x, '<', rhs, name="Bad Critter Percentage")
m.update()

#environmnet A harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,0]=-0.4
A[0,2]=-0.4
A[0,4]=1
A[0,5]=1
A[0,8]=1
A[0,9]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment A relationship with mix treated")
m.update()

#environmnet B harvest = amount treatment relationship contraint 
A = np.zeros([1,num_vars])
A[0,1]=-0.7
A[0,3]=-0.7
A[0,6]=1
A[0,7]=1
A[0,10]=1
A[0,11]=1
rhs = [0]
m.addMConstr(A , x, '=', rhs, name="Envornment B relationship with mix treated")
m.update()